# LDA topic modeling with a curated dataset

This example uses the [`gensim`](https://radimrehurek.com/gensim/index.html) library for building the topic model.

Depending on your collection size, this example will take between 10 minutes to an hour+ to run. 

In [1]:
import os

import warnings
warnings.filterwarnings('ignore')

In [2]:
import gensim
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

import pyLDAvis.gensim

pyLDAvis.enable_notebook()

stop_words = set(stopwords.words('english'))

Initialize a dataset object. 

In [3]:
from tdm_client import Dataset

dset = Dataset('a517ef1f-0794-48e4-bea1-ac4fb8b312b4')

In [4]:
len(dset)

1000

In [5]:
dset.query()

'q=shakespeare&start=0&rows=20&fq=yearPublished%3A%5B1900%20TO%202019%5D&fq=category%3A(%22Literature%20(General)%22%20OR%20%22English%20literature%22)'

Define a function for processing tokens from the extracted features for volumes in the curated dataset. This function:

* lowercases all tokens
* discards all tokens less than 4 characters
* discards non alphabetical tokens - e.g. --9
* removes stopwords using NLTK's stopword list
* Lemmatizes the token using NLTK's [WordNetLemmatizer](https://www.nltk.org/_modules/nltk/stem/wordnet.html)

In [6]:
def process_token(token):
    token = token.lower()
    if len(token) < 4:
        return
    if not(token.isalpha()):
        return
    if token in stop_words:
        return
    return WordNetLemmatizer().lemmatize(token)

Loop through the volumes in the dataset and make a list of tokens for each volume and then add to a list of the 25 documents in the dataset. We are limiting this example to 25 documents to limit the time it takes to run during demonstrations.

In [7]:
documents = []

for doc_n, volume in enumerate(dset.get_features()):
    this_doc = []
    try:
        pages = volume['features']['pages']
    except:
        continue
    for pn, page in enumerate(pages):
        body = page.get('body')
        if body is not None:
            for token, pos_count in body.get('tokenPosCount', {}).items():
                t = process_token(token)
                if t is None:
                    continue
                for pos, n in pos_count.items():
                    this_doc += [t] * n
    documents.append(this_doc)
    if doc_n >= 25:
        break
                

Build a gensim dictionary and corpus and then train the model.

In [8]:
dictionary = gensim.corpora.Dictionary(documents)

In [12]:
dictionary.filter_extremes(no_below=len(documents) * .10, no_above=0.5)

bow_corpus = [dictionary.doc2bow(doc) for doc in documents]


# train model, this might take some time
model = gensim.models.LdaModel(
    corpus=bow_corpus,
    id2word=dictionary,
    num_topics=10,
    passes=15
)


Visualize the model using [`pyLDAvis`](https://pyldavis.readthedocs.io/en/latest/).

In [13]:
pyLDAvis.gensim.prepare(model, bow_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.023069  0.124147       1        1  42.819485
8     -0.058272  0.069126       2        1  16.182970
5      0.008869  0.094555       3        1  12.736067
7      0.074418 -0.061600       4        1  12.672776
9      0.037052  0.048511       5        1   4.017588
2      0.197239 -0.034632       6        1   3.781209
4     -0.143145 -0.163749       7        1   2.928504
6      0.034688 -0.121118       8        1   2.437710
0     -0.114547  0.029368       9        1   2.422141
1     -0.013232  0.015392      10        1   0.001558, topic_info=      Category        Freq           Term       Total  loglift  logprob
3699   Default  123.000000         hawkes  123.000000  30.0000  30.0000
3432   Default  116.000000        derrida  116.000000  29.0000  29.0000
7933   Default  207.000000           arab  207.000000  28.0000  28.0000
1604   Default  496.000000       lighting  496.000000  27.0000  27.0000
985    Default   96.000000        epitaph   96.000000  26.0000  26.0000
4634   Default  164.000000          wilde  164.000000  25.0000  25.0000
6131   Default   82.000000      augustine   82.000000  24.0000  24.0000
3698   Default  149.000000       hathaway  149.000000  23.0000  23.0000
9513   Default   81.000000      nietzsche   81.000000  22.0000  22.0000
4874   Default   79.000000  commemoration   79.000000  21.0000  21.0000
7750   Default  140.000000        siddons  140.000000  20.0000  20.0000
7511   Default  137.000000         kemble  137.000000  19.0000  19.0000
7054   Default  118.000000           univ  118.000000  18.0000  18.0000
5720   Default  405.000000      repertory  405.000000  17.0000  17.0000
1557   Default   68.000000           kott   68.000000  16.0000  16.0000
4862   Default   96.000000      coleridge   96.000000  15.0000  15.0000
281    Default  263.000000    bolingbroke  263.000000  14.0000  14.0000
5897   Default   61.000000   tercentenary   61.000000  13.0000  13.0000
1520   Default  134.000000           joan  134.000000  12.0000  12.0000
3827   Default  101.000000         jesuit  101.000000  11.0000  11.0000
6354   Default  106.000000           eden  106.000000  10.0000  10.0000
10319  Default   72.000000           frye   72.000000   9.0000   9.0000
10298  Default   54.000000           ezra   54.000000   8.0000   8.0000
4494   Default   59.000000        terence   59.000000   7.0000   7.0000
4702   Default   95.000000    apocalyptic   95.000000   6.0000   6.0000
8141   Default  115.000000          egypt  115.000000   5.0000   5.0000
4062   Default  109.000000         online  109.000000   4.0000   4.0000
6357   Default  109.000000       egyptian  109.000000   3.0000   3.0000
4641   Default   81.000000     wordsworth   81.000000   2.0000   2.0000
4503   Default   60.000000       theology   60.000000   1.0000   1.0000
...        ...         ...            ...         ...      ...      ...
5236   Topic10    0.000250          harry  121.096153  -2.0218  -9.1284
9735   Topic10    0.000179        detract    3.340082   1.2368  -9.4604
11906  Topic10    0.000222          stint   38.753887  -1.0000  -9.2459
4714   Topic10    0.000249          ariel  162.024811  -2.3185  -9.1340
8841   Topic10    0.000180  deconstructed    3.414315   1.2168  -9.4584
4634   Topic10    0.000243          wilde  164.183273  -2.3562  -9.1583
9441   Topic10    0.000236        guthrie  114.027214  -2.0181  -9.1848
91     Topic10    0.000238         angelo  153.643295  -2.3101  -9.1786
6131   Topic10    0.000227      augustine   82.456245  -1.7339  -9.2248
6508   Topic10    0.000218       hercules   47.797272  -1.2269  -9.2630
9513   Topic10    0.000226      nietzsche   81.445641  -1.7253  -9.2285
3432   Topic10    0.000232        derrida  116.009705  -2.0552  -9.2047
6944   Topic10    0.000224       spelling   74.289818  -1.6440  -9.2391
1287   Topic10    0.000235         helena  146.722229  -2.2770  -9.19